# Генсим и остальные неприятности

## Импорты и подготовка

In [4]:
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

import re
import numpy as np
from pprint import pprint
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ознакомимся с нашим датасетом - загрузим его в панды, по красивой пандовской традиции всяких датасайентистов назовем датафреймом.

что у нас в его верхушке?

In [5]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## Очистка данных

Почистим эти данные

In [6]:
data = df.content.values.tolist()

data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

data = [re.sub('\s+', ' ', sent) for sent in data]

data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


парсим предложения до уровня слов

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Генсим

Мы дошли до генсима! Пойдем попросим его скрафтить биграммы и триграммы

In [8]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


Теперь создадим функции для обращения к генсиму

In [9]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


Мы готовим обучающий датасет!

In [11]:
id2word = corpora.Dictionary(data_lemmatized)

texts = data_lemmatized

corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


А теперь создаем функцию, которая найдет нам лучшие параметры

In [12]:
def choose_score(start, number, rate, corpus, id2word, data_lemmatized):
    best_coh = 0
    best_num = 0
    best_top = ''
    for i in range(start, number, rate):
        print(i)
        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

        coherence_lda_model = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
        coherence_lda = coherence_lda_model.get_coherence()
        if coherence_lda > best_coh:
            best_coh = coherence_lda
            best_num = i
            best_model = lda_model
            print(best_coh, best_num)

    return (best_coh, best_num, best_model)

In [13]:
choose_score(5, 30, 5, corpus, id2word, data_lemmatized)

5
0.4297369063851419 5
10
0.4973497938960755 10
15
20
25


(0.4973497938960755, 10, <gensim.models.ldamodel.LdaModel at 0x7fbf26ceac88>)

Сама функция для генсимовской модели!

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
doc_lda = lda_model[corpus]
t = lda_model.print_topics()

In [18]:
list(t)

[(0,
  '0.028*"team" + 0.027*"year" + 0.026*"game" + 0.020*"play" + 0.018*"win" + 0.014*"player" + 0.010*"run" + 0.010*"last" + 0.009*"good" + 0.009*"hit"'),
 (1,
  '0.022*"go" + 0.015*"time" + 0.013*"day" + 0.012*"come" + 0.011*"take" + 0.011*"back" + 0.011*"get" + 0.009*"say" + 0.008*"see" + 0.008*"first"'),
 (2,
  '0.014*"space" + 0.008*"cost" + 0.008*"year" + 0.007*"high" + 0.007*"research" + 0.007*"low" + 0.006*"item" + 0.006*"also" + 0.006*"test" + 0.005*"large"'),
 (3,
  '0.033*"car" + 0.021*"drive" + 0.013*"bike" + 0.011*"power" + 0.011*"wire" + 0.011*"slave" + 0.010*"reality" + 0.009*"speed" + 0.009*"engine" + 0.009*"light"'),
 (4,
  '0.090*"ax" + 0.077*"max" + 0.018*"di_di" + 0.015*"tumor" + 0.012*"homosexual" + 0.011*"gay" + 0.009*"taste" + 0.008*"liar" + 0.007*"marry" + 0.006*"homosexuality"'),
 (5,
  '0.017*"government" + 0.013*"people" + 0.012*"gun" + 0.011*"state" + 0.010*"kill" + 0.008*"year" + 0.007*"public" + 0.007*"attack" + 0.007*"patient" + 0.007*"country"'),
 (6,


Мы дошли до определения списка слов в топиках!

In [19]:
topic_dict = {}
for topic in list(t):
    topic = tuple(topic)
    new_top = {}
    for word in topic[1].split(' + '):
        word = word.split('*')
        new_top[word[1].strip('\"\"')] = float(word[0])
    topic_dict[topic[0]] = new_top

In [20]:
topic_dict[0]

{'game': 0.026,
 'good': 0.009,
 'hit': 0.009,
 'last': 0.01,
 'play': 0.02,
 'player': 0.014,
 'run': 0.01,
 'team': 0.028,
 'win': 0.018,
 'year': 0.027}

Это был список слов для топика спорт? Как мило

In [21]:
text_topics = []
for text in data_lemmatized:
    count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for word in text:
        for topic, lemmas in topic_dict.items():
            for lemm in lemmas:
                if lemm == word:
                    count[topic] += lemmas[lemm]
    text_topics.append(count.index(max(count)))

In [22]:
if len(text_topics)==len(data_lemmatized):
  print('Carry on')

Carry on


Порядок, ничего не потерялось

## ТФ-ИДФ

In [23]:
df_prefinal = pd.DataFrame()
df_prefinal['texts'] = df['content']
df_prefinal['lemmas'] = data_lemmatized
df_prefinal['topics'] = text_topics
df_prefinal['ind'] = [x for x in range(0,11314)]

In [24]:
df_prefinal.head()

,texts,lemmas,topics,ind
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,"[where, thing, car, nntp_poste, host, park, li...",3,0
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,"[poll, final, call, summary, final, call, cloc...",6,1
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,"[engineering, computer, network, distribution_...",7,2
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,"[division, line, host, write, write, article, ...",7,3
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,"[question, distribution, article, write, clear...",7,4


Как красиво! У нас есть варианты топиков, леммы, рабочая модель!

по такой радости выведем сообщения из первого топика

In [25]:
df_1 = df_prefinal[df_prefinal['topics'] == 0]

In [26]:
df_1.head()

,texts,lemmas,topics,ind
17,From: CPKJP@vm.cc.latech.edu (Kevin Parker)\nS...,"[insurance, rate, performance, car, line, late...",0,17
43,From: paul@csd4.csd.uwm.edu (Paul R Krueger)\n...,"[rock, division, line, nntp_poste, host, origi...",0,43
49,From: 18084TM@msu.edu (Tom)\nSubject: Golden &...,"[age, added_forwarde, organization, distributi...",0,49
208,From: seth@cbnewsh.cb.att.com (peter.r.clark.....,"[flyer, year, big, worst_opinion, organization...",0,208
256,From: dpeterik@iastate.edu (Dan Peterik)\nSubj...,"[line, write, know, brewer, s, back, team, mas...",0,256


In [27]:
df_1['lemmas'].tolist()[0]

['insurance',
 'rate',
 'performance',
 'car',
 'line',
 'latech',
 'newsreader_nnr',
 'recently',
 'post',
 'article',
 'ask',
 'kind',
 'rate',
 'single',
 'male',
 'driver',
 'yrs',
 'old',
 'pay',
 'performance',
 'car',
 'here',
 'summary',
 'reply',
 'receive',
 'be',
 'anymore',
 'close',
 'enough',
 'twin',
 'turbo',
 'model',
 'ticket',
 'accident',
 'take',
 'defensive',
 'driving',
 'airbag',
 'security',
 'single',
 'year',
 'state',
 'farm',
 'insurance',
 'include',
 'additional',
 'umbrella',
 'policy',
 'car',
 'house',
 'base',
 'policy',
 'standard',
 'policy',
 'require',
 'defensive',
 'drive',
 'course',
 'less',
 'buy',
 'car',
 'company',
 'never',
 'accident',
 'ticket',
 'year',
 'quote',
 'hope',
 'help',
 'remember',
 'name',
 'correctly',
 'ask',
 'insurance',
 'performance',
 'car',
 'well',
 'last',
 'year',
 'similar',
 'situation',
 'buy',
 'car',
 'make',
 'inquiry',
 'age',
 'car',
 'drive',
 'record',
 'turn',
 'may',
 'insurance',
 'go',
 'mos',
 'al

Звучит как правдоподобный вариант слов, по которым можно ориентироваться, чтобы спихать куда надо объекты про экономику

In [28]:
def findwords(vect, words):
    sorted_w = np.argsort(vect.data)[:-6:-1]
    return words[vect.indices[sorted_w]]

In [29]:
all_topics = {}
for i in range(10):
    list_text = []
    dict_of_topic = {}
    df_1 = df_prefinal[df_prefinal['topics'] == i]
    lemmas = df_1['lemmas'].tolist()
    inds = df_1['ind'].tolist()
    for num, t in enumerate(inds):
        dict_of_topic[t] = lemmas[num]
        list_text.append(' '.join(lemmas[num]))
    vectors = vectorizer.fit_transform(list_text)
    feature_names = np.array(vectorizer.get_feature_names())
    #dense = vectors.todense()
    #denselist = dense.tolist()
    #df = pd.DataFrame(denselist, columns=feature_names)
    3#for text in df.index.tolist():
        #a = df[df[index] == text].tolist

    for index_lemm, l in enumerate(lemmas):
        #tfidf_dict = {}
        text_vect = vectors[index_lemm, :]
        top_words = findwords(text_vect, feature_names)
        all_topics[inds[index_lemm]] = top_words


In [30]:
list(all_topics[2])

['display', 'powerbook', 'machine', 'bunch', 'disk']

In [31]:
topics_list = []
for key in range(0, 11314):
    topics_list.append(all_topics[key])

In [32]:
df_prefinal['topwords'] = topics_list

In [33]:
df_prefinal

,texts,lemmas,topics,ind,topwords
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,"[where, thing, car, nntp_poste, host, park, li...",3,0,"[door, car, where, enlighten, funky]"
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,"[poll, final, call, summary, final, call, cloc...",6,1,"[poll, clock, final, upgrade, add]"
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,"[engineering, computer, network, distribution_...",7,2,"[display, powerbook, machine, bunch, disk]"
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,"[division, line, host, write, write, article, ...",7,3,"[division, quadrilateral, chip, weitek, winter]"
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,"[question, distribution, article, write, clear...",7,4,"[error, warn, bug, memory, waivere]"
...,...,...,...,...,...
11309,From: jim.zisfein@factory.com (Jim Zisfein) \n...,"[scan, city, reply, line, consultation, cheap,...",7,11309,"[patient, tumor, diagnosis, dn, scan]"
11310,From: ebodin@pearl.tufts.edu\nSubject: Screen ...,"[screen, medford, old, problem, screen, blank,...",1,11310,"[screen, blank, wire, new, board]"
11311,From: westes@netcom.com (Will Estes)\nSubject:...,"[este, mount, mail, group, line, instal, try, ...",6,11311,"[cpu, mount, este, weight, cool]"
11312,From: steve@hcrlgw (Steven Collins)\nSubject: ...,"[line, article, write, boy, embarasse, trivial...",7,11312,"[point, space, farin, specifie, embarasse]"


Мы рассортировали все по темам!
Вуаля